In [1]:
import os

from keras import backend as K
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras.datasets import cifar100
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

os.environ['KERAS_BACKEND'] = 'tensorflow'

In [ ]:
import freezing
import densenet

In [5]:
batch_size = 64
nb_classes = 100
nb_epoch = 300

img_rows, img_cols = 32, 32
img_channels = 3

img_dim = (img_channels, img_rows, img_cols) if K.image_data_format() == "channels_first" else (img_rows, img_cols, img_channels)
depth = 40
nb_dense_block = 3
growth_rate = 12
bottle_neck = False
nb_filter = 12
reduction = 0.0
dropout_rate = 0.0 # 0.0 for data augmentation
seed = 333
weight_decay = 0.0001
learning_rate = 0.1

# Preprocessing for DenseNet https://arxiv.org/pdf/1608.06993v3.pdf
def color_preprocessing(x_train,x_test):
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    mean = [125.307, 122.95, 113.865]
    std  = [62.9932, 62.0887, 66.7048]
    for i in range(3):
        x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
        x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]
    return x_train, x_test

In [ ]:
model = densenet.DenseNet(img_dim, classes=nb_classes, depth=depth, nb_dense_block=nb_dense_block,
                          growth_rate=growth_rate, nb_filter=nb_filter, dropout_rate=dropout_rate, weights=None, weight_decay=1e-4)
print("Model created")

model.summary()
sgd = SGD(learning_rate=0.1, momentum=0.9, nesterov=True)  # dampening = 0.9? Should be zero?
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=["accuracy"])
print("Finished compiling")
print("Building model...")

(x_train, y_train), (x_test, y_test) = cifar100.load_data()

#For data preprocessing, we normalize the data using the channel means and standard deviations (https://arxiv.org/pdf/1608.06993v3.pdf)
x_train, x_test = color_preprocessing(x_train, x_test)

x_train45, x_val, y_train45, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=seed)  # random_state = seed


img_gen = ImageDataGenerator(
    horizontal_flip=True,
    width_shift_range=0.125,  # 0.125*32 = 4 so max padding of 4 pixels, as described in paper.
    height_shift_range=0.125,  # first zero-padded with 4 pixels on each side, then randomly cropped to again produce 32×32 images
    fill_mode = "constant",
    cval = 0
)

y_train45 = to_categorical(y_train45, nb_classes)  # 1-hot vector
y_val = to_categorical(y_val, nb_classes)
y_test = to_categorical(y_test, nb_classes)

img_gen.fit(x_train45, seed=seed)

In [7]:
freezing_list = []
for i in range(len(model.layers)):
  if i < len(model.layers) * 0.8:
    freezing_list.append(int(nb_epoch*0.6))
freezing_list.append(nb_epoch)
checkpointer = ModelCheckpoint('model_cont_dense_c100_best.hdf5', verbose=1, save_best_only=True)

In [8]:
freezing.training_with_freezing(model, img_gen, sgd, x_train45, y_train45, x_val, y_val, x_test, y_test,freezing_list,batch_size=batch_size,lr_schedule = [[0, 0.1],[nb_epoch*0.5,0.01],[nb_epoch*0.75,0.001]], cbks=[checkpointer],name='dense_cifar100')

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......activation
.........vars
......activation_1
.........vars
......activation_10
.........vars
......activation_11
.........vars
......activation_12
.........vars
......activation_13
.........vars
......activation_14
.........vars
......activation_15
.........vars
......activation_16
.........vars
......activation_17
.........vars
......activation_18
.........vars
......activation_19
.........vars
......activation_2
.........vars
......activation_20
.........vars
......activation_21
.........vars
......activation_22
.........vars
......activation_23
.........vars
......activation_24
.........vars
......activation_25
.........vars
......activation_26
.........vars
......activation_27
.........vars
......activation_28
.........vars
......activation_29
.........vars
......activation_3
.........vars
......activation_30
.........vars
......activation_31
.........vars
......activation_32
.........vars
......activa

[30.019999999999996,
 0.20820267888903615,
 0.4635832021871118,
 2.0276901432766756,
 0.2740956561416483]